<a href="https://colab.research.google.com/github/swissecon-cryptecon/public_blockchains/blob/main/Public_Blockchains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install messari

You should consider upgrading via the 'C:\Users\MatthiasHafner\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [2]:
from messari.messari import Messari
import pandas as pd

In [3]:
# Messari API Key
messari = Messari('OwwZ+mQ92hTCnYuzB-Gx9+927B2h-PNBIiyihjsVMmnF3to4')

In [29]:
#list of variables, see also documentation: https://messari.io/api/docs#tag/News/operation/Get%20News%20for%20Asset
#print(dir(messari)) 
list_messari_metrics = messari.list_asset_timeseries_metric_ids()
data = list_messari_metrics["data"]["metrics"]  
df = pd.DataFrame(data)
df.to_csv("messari_metrics.csv", index=False)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_get', '_send_message', 'get_all_assets', 'get_all_markets', 'get_all_news', 'get_asset', 'get_asset_market_data', 'get_asset_metrics', 'get_asset_profile', 'get_asset_timeseries', 'get_market_timeseries', 'get_news_for_asset', 'key', 'list_asset_timeseries_metric_ids', 'session']


In [126]:
asset = "solana"
metric = "act.addr.cnt"
start = '2017-01-01'
end = '2023-04-15'
interval = '1w'

query_params = {
    'start': start,
    'end': end,
    'interval': interval,
    'order': 'ascending',
    'format': 'json',
    'timestamp-format': 'rfc3339'
}
response = messari.get_asset_timeseries(asset_key=asset, metric_id=metric, **query_params)
response

{'status': {'elapsed': 17,
  'timestamp': '2023-04-16T13:55:23.442314905Z',
  'error_code': 404,
  'error_message': 'Not Found'}}

In [164]:
# Retrieve historical timeseries data for an asset.
def messari_query(asset, metric, start = '2017-01-01', end = '2023-04-15', interval = '1w'):

    query_params = {
        'start': start,
        'end': end,
        'interval': interval,
        'order': 'ascending',
        'format': 'json',
        'timestamp-format': 'rfc3339'
    }
    response = messari.get_asset_timeseries(asset_key=asset, metric_id=metric, **query_params)
    error_code = response["status"].get("error_code", None)
    error_msg = response['status'].get("error_message", None)

    if error_code is not None:
        error_msg = asset + ", " + metric + ": Error " +  error_msg
        print(error_msg)
        print("Metric '" + metric + "' for " + asset + " is ignored!")
        df = pd.DataFrame()
        df['timestamp'] = None
        df['asset'] = None
    else:
        # Assume the output dictionary is stored in a variable called 'output'
        data = response['data']
        values = data['values']

        # Convert the values list to a Pandas DataFrame
        df = pd.DataFrame(values, columns=data['schema']['values_schema'].keys())

        # Add a column for the asset name
        df['asset'] = asset

        # Convert the timestamp column to a datetime data type
        df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Print the resulting DataFrame
    return df

In [165]:
def multi_messari_query(assets, metrics, start = '2020-01-01', end = '2023-04-15', interval = '1w'):
    #initialize empty dataframe
    df_merged = pd.DataFrame()
    df_merged['timestamp'] = None
    df_merged['asset'] = None

    #loop through assets and metrics
    for asset in assets:
        df_merged_metric = pd.DataFrame()
        df_merged_metric['timestamp'] = None
        df_merged_metric['asset'] = None
        for metric in metrics:
            df = messari_query(asset, metric, start = start, end = end, interval = interval)
            df_merged_metric = pd.merge(df_merged_metric, df, how='outer', on=['timestamp', 'asset'])  ### do we loose obs with this merge?
        
        df_merged = pd.concat([df_merged, df_merged_metric], axis=0)

    return df_merged

In [163]:
# example singe messari query
asset = 'solana'
metric = 'act.addr.cnt'  ##  'mcap.circ', mcap.dom  ## circulating market supply
df1, error_msg = messari_query(asset, metric)
print(error_msg)


solana, act.addr.cnt: Error Not Found
Metric 'act.addr.cnt' for solana is ignored.
asset


In [167]:
#exampe multi messari query
assets = ["bitcoin", "ethereum", "cardano", "polkadot", "solana"] # no data on addresses solona --> error
metrics = ["mcap.circ", "act.addr.cnt", "mcap.dom"] # circulating_mar
df = multi_messari_query(assets=assets,metrics=metrics)
df

solana, act.addr.cnt: Error Not Found
Metric 'act.addr.cnt' for solana is ignored!


,timestamp,asset,circulating_marketcap,active_addresses,marketcap_dominance
0,2020-01-06 00:00:00+00:00,bitcoin,1.472557e+11,581504.0,69.994370
1,2020-01-13 00:00:00+00:00,bitcoin,1.606576e+11,622774.0,68.389125
2,2020-01-20 00:00:00+00:00,bitcoin,1.538085e+11,544915.0,67.975179
3,2020-01-27 00:00:00+00:00,bitcoin,1.703184e+11,706042.0,67.367427
4,2020-02-03 00:00:00+00:00,bitcoin,1.835028e+11,695691.0,65.624116
...,...,...,...,...,...
153,2023-03-13 00:00:00+00:00,solana,8.151069e+09,NaN,0.703298
154,2023-03-20 00:00:00+00:00,solana,7.841841e+09,NaN,0.674893
155,2023-03-27 00:00:00+00:00,solana,8.131239e+09,NaN,0.675096
156,2023-04-03 00:00:00+00:00,solana,7.780007e+09,NaN,0.651450


In [168]:
fig = px.line(df, x="timestamp", y=["active_addresses"], color="asset").update_layout(height=500)   #, facet_row="subset"
fig.show()

In [169]:
import plotly.express as px

fig = px.line(df, x="timestamp", y=["marketcap_dominance"], color="asset").update_layout(height=500)   #, facet_row="subset"
fig.show()

In [171]:
df_last = df.groupby('asset').last().reset_index()

In [172]:
df_last

,asset,timestamp,circulating_marketcap,active_addresses,marketcap_dominance
0,bitcoin,2023-04-10 00:00:00+00:00,5.483222e+11,947327.0,45.521603
1,cardano,2023-04-10 00:00:00+00:00,1.354891e+10,51215.0,1.124828
2,ethereum,2023-04-10 00:00:00+00:00,2.240408e+11,548021.0,18.599825
3,polkadot,2020-08-10 00:00:00+00:00,7.587513e+09,1303.0,0.629914
4,solana,2023-04-10 00:00:00+00:00,7.881990e+09,NaN,0.654361


In [173]:
fig = px.pie(df, values='marketcap_dominance', names='asset')
fig.show()